# String Transformations

In [1]:
!pip install more-dfply

In [2]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [3]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Outline

1. Basic string operations<br>
    a. Managing case and whitespace<br>
    b. Replacing substrings<br>
2. Splitting strings and `dfply.separate`
3. Concatenating columns and `dfply.unite`
4. Regular expressions in Python
5. Using functions and RegEx with `df.column.replace`

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

## The exhibition file gives encoding errors by default

In [4]:
exhibitions = pd.read_csv('./data/MoMAExhibitions1929to1989.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 405: invalid continuation byte

## Switching encodings fixes the problem

* See [this Stack Overflow question](https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python)
* More details on [ISO-8859-1](https://en.wikipedia.org/wiki/ISO/IEC_8859-1)

In [5]:
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate', 'ConstituentBeginDate' ,'ConstituentEndDate']
exhibitions = pd.read_csv('./data/MoMAExhibitions1929to1989.csv', 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(2)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902,1981,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839,1906,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053


#### MoMA Artists

In [6]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


#### MoMA Artwork

In [8]:
from more_dfply import fix_names

artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


# Working with Strings

In [9]:
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


## Working with string  `pandas`

* Available under `df.column.str`
* Match most python methods

## Managing case and whitespace

The following table compare the functions/methods for managing case

| `python` method | `pandas` method  |
| --- | --- |
| `s.lower()` | `df.a.str.lower()`   |
| `s.upper()` | `df.a.str.upper()`   |
| `s.strip()` | `df.a.str.strip()`   |
| `s.lstrip()` | `df.a.str.lstrip()` |
| `s.rstrip()` | `df.a.str.rstrip()` |

In [10]:
[m for m in dir(artwork.Artist.str) if not m.startswith('_')]

['capitalize',
 'casefold',
 'cat',
 'center',
 'contains',
 'count',
 'decode',
 'encode',
 'endswith',
 'extract',
 'extractall',
 'find',
 'findall',
 'fullmatch',
 'get',
 'get_dummies',
 'index',
 'isalnum',
 'isalpha',
 'isdecimal',
 'isdigit',
 'islower',
 'isnumeric',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'len',
 'ljust',
 'lower',
 'lstrip',
 'match',
 'normalize',
 'pad',
 'partition',
 'removeprefix',
 'removesuffix',
 'repeat',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'slice',
 'slice_replace',
 'split',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'wrap',
 'zfill']

## Example - Lower-case Artists

In [12]:
artwork.Artist.str.lower()

0                                               otto wagner
1                                  christian de portzamparc
2                                                emil hoppe
3                                           bernard tschumi
4                                                emil hoppe
                                ...                        
138146    chesnutt brothers studio, andrew chesnutt, lew...
138147                                   sophie taeuber-arp
138148                                   sophie taeuber-arp
138149                                   sophie taeuber-arp
138150                                   sophie taeuber-arp
Name: Artist, Length: 138151, dtype: object

#### Applying `str.lower` in `pandas`

In [13]:
(artwork
 >> select(X.Artist)
 >> mutate(lower_artist = X.Artist.str.lower())
 >> head(2)
)

,Artist,lower_artist
0,Otto Wagner,otto wagner
1,Christian de Portzamparc,christian de portzamparc


## Replacing a substring

Another important string operation involves replacing one substring with another.  In this section, we will illustrate using a regular expression to accomplish this task.

## Example - The BeginDate mess

In [14]:
artwork.BeginDate.head()

0    (1841)
1    (1944)
2    (1876)
3    (1944)
4    (1876)
Name: BeginDate, dtype: object

## Using replace to remove the left parenthesis

In [15]:
# Currently (2022) results in a warning
(artwork.BeginDate
 .str.replace('(', '')
 .head()
)

/tmp/ipykernel_851/2367958832.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  (artwork.BeginDate


0    1841)
1    1944)
2    1876)
3    1944)
4    1876)
Name: BeginDate, dtype: object

In [16]:
# specifying regex = False/True removes the warning
(artwork.BeginDate
 .str.replace('(', '', regex=False)
 .head()
)

0    1841)
1    1944)
2    1876)
3    1944)
4    1876)
Name: BeginDate, dtype: object

###### Dot chaining two replace calls

**Note:** Recall you need a `.str.` before each string method

In [17]:
(artwork.BeginDate
 .str.replace('(', '', regex=False)
 .str.replace(')', '', regex=False)
 .head()
)

0    1841
1    1944
2    1876
3    1944
4    1876
Name: BeginDate, dtype: object

## Using a regular expression

Recall that `pandas` method accepts [regular expressions](https://en.wikipedia.org/wiki/Regular_expression), which can be used to replace one of any number of characters.

In [18]:
(artwork.BeginDate
 .str.replace('[()]', '', regex=True)
 .head()
)

0    1841
1    1944
2    1876
3    1944
4    1876
Name: BeginDate, dtype: object

# Splitting strings

* `split` - important string tool
* Using `split` on a column returns a column of lists
* `pandas`: `df.c.split(pattern)` returns an object class containing a lists
* Accepts regular expressions

### Example 1 - Splitting the Artists Name

In [19]:
artwork.Artist.str.split(' ').head(2)

0                  [Otto, Wagner]
1    [Christian, de, Portzamparc]
Name: Artist, dtype: object

## `dtype: object` - more than just a string

<img src="./img/split_object.png" width=400>

While `dtype: object` is usually a `str`, this time we get a `list`.

## Using `get` to get one part

Since `split` returns a list, we will need to use `get` to pull out an entry.

In [20]:
(artwork.Artist
 .str.split(' ')
 .str.get(0)
 .head()
)

0         Otto
1    Christian
2         Emil
3      Bernard
4         Emil
Name: Artist, dtype: object

## Remember negative indexing!

* Negative index == counting from the right

In [21]:
(artwork.Artist
 .str.split(' ')
 .str.get(-1) # Get last
 .head()
)

0         Wagner
1    Portzamparc
2          Hoppe
3        Tschumi
4          Hoppe
Name: Artist, dtype: object

### Example 2 - Splitting the Artists Bio

In [24]:
(artwork
 >> select(X.ArtistBio)
 >> mutate(ArtistBio = (X.ArtistBio
                        .str.replace('[()]', '', regex = True)
                       )
          )
 >> mutate(ArtistiBioNew = (X.ArtistBio
                            .str.split(r' ,|, born|-')
                           )
          )
).head()

,ArtistBio,ArtistiBioNew
0,"Austrian, 1841–1918","[Austrian, 1841–1918]"
1,"French, born 1944","[French, 1944]"
2,"Austrian, 1876–1957","[Austrian, 1876–1957]"
3,"French and Swiss, born Switzerland 1944","[French and Swiss, Switzerland 1944]"
4,"Austrian, 1876–1957","[Austrian, 1876–1957]"


## <font color="red"> Exercise 3.1.1</font>

1. Why didn't splitting on `-` work in the last example?
2. See if you can fix this issue

> There are different types of hyphens, so being careful about which to use is essential

In [25]:
# Your code here
(artwork
 >> select(X.ArtistBio)
 >> mutate(ArtistBio = (X.ArtistBio
                        .str.replace('[()]', '', regex = True)
                       )
          )
 >> mutate(ArtistiBioNew = (X.ArtistBio
                            .str.split(r' ,|, born|–')
                           )
          )
).head()

,ArtistBio,ArtistiBioNew
0,"Austrian, 1841–1918","[Austrian, 1841, 1918]"
1,"French, born 1944","[French, 1944]"
2,"Austrian, 1876–1957","[Austrian, 1876, 1957]"
3,"French and Swiss, born Switzerland 1944","[French and Swiss, Switzerland 1944]"
4,"Austrian, 1876–1957","[Austrian, 1876, 1957]"


## Extracting a substring of a fixed size/position

In `pandas`, use the `str.slice` method
* Use `start=` to specify the optional starting place
* Use `stop=` to specify the optional stopping location


In [26]:
(artwork
 >> select(X.BeginDate) 
 >> mutate(BeginDate = (X.BeginDate
                        .str.replace('[()]', '', regex = True)))
 >> mutate(century = (X.BeginDate
                      .str.slice(stop = 2)),
           year_in_century = (X.BeginDate
                              .str.slice(start = 2)))
 >> head(5))

,BeginDate,century,year_in_century
0,1841,18,41
1,1944,19,44
2,1876,18,76
3,1944,19,44
4,1876,18,76


## Extracting a substring with RegEx

To extract a column in `pandas` with RegEx

* Use `more_dfply.extract(col, pat)` with exactly one group
* `pat` should be a regular expression with exactly one group

In [27]:
from more_dfply import extract

(artwork
 >> select(X.ArtistBio)
 >> mutate(country_of_birth = extract(X.ArtistBio, r', born ([a-zA-Z]+)'),
           year_of_death = extract(X.ArtistBio, r'–(\d{4,4})'))
 >> head(5))

,ArtistBio,country_of_birth,year_of_death
0,"(Austrian, 1841–1918)",NaN,1918
1,"(French, born 1944)",NaN,NaN
2,"(Austrian, 1876–1957)",NaN,1957
3,"(French and Swiss, born Switzerland 1944)",Switzerland,NaN
4,"(Austrian, 1876–1957)",NaN,1957


## <font color="red"> Exercise 3.1.2 </font>

**Tasks:**

1. Inspect the `group` column, which contains both the sex and age.  Identify the pattern and develop/test a RegEx.
2. Use RegEx to extract the following columns
    a. Gender
    b. age_lower_limit
    c. age_upper_limit

Example based on an example from [Wickim (2014)](https://vita.had.co.nz/papers/tidy-data.pdf)

In [38]:
df = pd.read_csv("./data/TB_bad_stacked.csv")
df.sample(5)

,country,group,cases
1839,LR,m0-14,1000.0
2564,TH,m25-34,9300.0
1942,ZW,m0-14,2900.0
2590,ZW,m25-34,5000.0
2469,JM,m25-34,NaN


In [41]:
# Your code here
(df
 >> mutate(gender = extract(X.group, r'(\w)'),
           age_lowr_limit = extract(X.group, r'\w(\d+)-'),
           age_uppr_limit = extract(X.group, r'\w\d+-(\d+)')
          )
 >> sample(5)
)

,country,group,cases,gender,age_lowr_limit,age_uppr_limit
2127,SY,m0-4,110.0,m,0,4
2480,KW,m25-34,NaN,m,25,34
2865,EE,m45-54,NaN,m,45,54
2916,LB,m45-54,NaN,m,45,54
1618,KZ,f55-64,410.0,f,55,64


## Recoding with a `dict` in `pandas`

* Use `more_dfply.recode(col, d, default=None)`
* `d` is the translation `dict`
* Use `default` keyword to add a default value

In [42]:
from more_dfply import recode

new_gender = {'Male':'m', 'Female':'f'}

(artists
 >> mutate(Gender = recode(X.Gender, new_gender))
 >> row_slice([13,14]))

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
13,16,Cristobal Arteche,"Spanish, 1900–1964",Spanish,m,1900,1964,NaN,NaN
14,18,Artko,NaN,NaN,NaN,0,0,NaN,NaN


In [43]:
(artists
 >> mutate(Gender = recode(X.Gender, new_gender, default='Unknown'))
 >> row_slice([13,14]))

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
13,16,Cristobal Arteche,"Spanish, 1900–1964",Spanish,m,1900,1964,NaN,NaN
14,18,Artko,NaN,NaN,Unknown,0,0,NaN,NaN


## <font color="red"> Exercise 3.1.3</font>

Use `recode` to create a column called `American` in the `artists` data set that will contains `'Yes'` if the artist is American and `'No'` otherwise.

In [47]:
# Your code here
IfAmerican = {'American':'Yes'}

(artists
 >> mutate(American = recode(X.Nationality, IfAmerican, default='No'))
 >> sample(5))

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN,American
8748,19962,Hans-Ulrich Obrist,"Swiss, born 1968",Swiss,Male,1968,0,NaN,NaN,No
12174,37779,Jeff Preiss,"American, born 1956",American,Male,1956,0,NaN,NaN,Yes
1927,2171,John Giorno,"American, 1936–2019",American,Male,1936,2019,Q642513,NaN,Yes
14755,68764,New Breed,"American, founded 1968",American,NaN,1968,0,NaN,NaN,Yes
392,447,Ross Bellah,"American, 1907–2004",American,Male,1907,2004,NaN,NaN,Yes


# `dfply` column splitting function

Next, we will look at two useful `dfply` functions, `separate` and `unite`

## Using `dfply.separate` to split and create columns 

* Many application of `split` return multiple columns
* Using `str.split` requires many calls to `dfply.mutate`
* `dfply.separate` allows you to get this in one fell swoop

## Example column

Clearly this column violates the Golden Rule. 

In [48]:
d = pd.DataFrame({'a':['1-a-3', '1-b', '1-c-3-4', '9-d-1', '10']})
d

,a
0,1-a-3
1,1-b
2,1-c-3-4
3,9-d-1
4,10


## Using separate with `fill='right'`

* Nothing to split in last row
* `fill='right'` $\rightarrow$ `NaN` is *filled in* on the right.

In [49]:
(d 
 >> separate(X.a, ['col1', 'col2'], 
             remove=True, convert=True,
            extra='drop', fill='right'))

,col1,col2
0,1,a
1,1,b
2,1,c
3,9,d
4,10,NaN


## Using separate with `fill='left'`

* Nothing to split in last row
* `fill='left'` $\rightarrow$ `NaN` on the left.

In [50]:
(d 
 >> separate(X.a, ['col1', 'col2'], 
             remove=True, convert=True,
            extra='drop', fill='left'))

,col1,col2
0,1.0,a
1,1.0,b
2,1.0,c
3,9.0,d
4,NaN,10


## Using separate with `extra='merge'`

* `extra='merge'`$\rightarrow$ puts the rest in another column
* No lost data

In [51]:
(d 
 >> separate(X.a, ['col1', 'col2'], 
             remove=False, convert=True,
              extra='merge', fill='right'))

,a,col1,col2
0,1-a-3,1,a-3
1,1-b,1,b
2,1-c-3-4,1,c-3-4
3,9-d-1,9,d-1
4,10,10,NaN


## Using separate with `extra='merge'`

* `sep=[2,4]` separates by position instead of splitting
* `extra='merge'`$\rightarrow$ only makes an extra column when needed.

In [54]:
(d 
 >> separate(X.a, ['col1', 'col2', 'col3'], 
             sep=[2,4], remove=True, convert=True,
             extra='merge', fill='right'))

,a,col1,col2,col3
0,1-a-3,1,a,3.0
1,1-b,1,b,NaN
2,1-c-3-4,1,c,3.0
3,9-d-1,9,d,1.0
4,10,10,NaN,NaN


## <font color="red"> Exercise 3.1.4 </font>

**Tasks:** Use `separate` to pull these data apart into three separate columns in one function call in the `TD_bad_stacked.csv` data.

In [53]:
df = pd.read_csv("./data/TB_bad_stacked.csv")
df.head()

,country,group,cases
0,AD,f0-14,0.0
1,AE,f0-14,4.0
2,AF,f0-14,3500.0
3,AG,f0-14,0.0
4,AI,f0-14,0.0


In [67]:
# Your code here
(df 
 >> separate(X.group, ['gender', 'lowr_age_lmt', 'uppr_age_lmt'], 
             sep=[1,3], remove=False, convert=True,
             extra='merge', fill='right')
 >> sample(5))

,country,group,cases,gender,lowr_age_lmt,uppr_age_lmt
1378,HR,f5-14,NaN,f,5-,14
2124,ST,m0-4,NaN,m,0-,4
1535,BM,f55-64,NaN,f,55,-64
1996,DK,m0-4,NaN,m,0-,4
320,KW,f0-4,NaN,f,0-,4
